In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time as T
import requests
from bs4 import BeautifulSoup
import json
import warnings
import re 
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import urllib.request
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np 
import os 
def sleep(time):
    return (T.sleep(time))

def getting_URL(driver,url):
    
    return(driver.get(url))


def login(driver,email,password):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, "email"))).send_keys(email)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, "password"))).send_keys(password)
    return(driver.find_element_by_class_name("login100-form-btn").click())

def hovering_dropdown_menue(driver,Xpath):
    dropdown_trigger =WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, Xpath)))
    return (ActionChains(driver).move_to_element(dropdown_trigger).perform())

def Clicking_on_element(driver,Xpath):
    dropdown_option = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,Xpath)))
    return (dropdown_option.click())

def Inserting_Entry(driver,Xpath,Entry):
    Element= WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,Xpath)))

    return (Element.send_keys(Entry) )


def clear_entire_text(driver,Xpath):
    
        element= WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,Xpath)))
        return (element.send_keys(Keys.CONTROL + 'a', Keys.BACKSPACE))


    
def country_inserting_add_account(driver,GTC_xpath,GTPW_xpath,COE_xpath,country_name):
    going_to_the_cell=Clicking_on_element(driver,GTC_xpath)

    going_to_the_place_to_write = WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, GTPW_xpath)))
    going_to_the_place_to_write.send_keys(country_name)
    return (Clicking_on_element(driver,COE_xpath))
 

def Clicking_chcek_box(driver,xpath):
        option=WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, xpath)))
        return (driver.execute_script("arguments[0].click();", option))
    

def Add_Quantity(number,Add=0,Mins=0):
    
    sleep(0.25)
    if Add=='Add':
        for i in range (0,number-1):
            Clicking_on_element(driver, '//*[@id="quantity_wanted_p"]/a[2]/span/i')
    
    
    if Mins=='Mins':
        for i in range (0,number-1):
            Clicking_on_element(driver, '//*[@id="quantity_wanted_p"]/a[1]/span')
        
def Choose_Color(Color):
    
    if Color =='orange':
        driver.find_element_by_name('Orange').click()
        
    if Color =='blue':
        driver.find_element_by_name('Blue').click() 
        
        
def Choose_size(type_size):
    
    drop_down_list=Clicking_on_element(driver, '//*[@id="group_1"]')
    Text_unique=type_size
    string_AI="//*[text()[contains(.,'"+str(Text_unique)+"')]]"

    if type_size =='S':
        Clicking_on_element(driver, '//*[@id="group_1"]/option[1]')

    if type_size =='M':
        Clicking_on_element(driver, '//*[@id="group_1"]/option[2]') 
        
    if type_size =='L':
        Clicking_on_element(driver, '//*[@id="group_1"]/option[3]') 
        
        
def Click_on_element_by_displayed_text(driver,text):
    string_AI="//*[text()[contains(.,'"+str(text)+"')]]"
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, string_AI))).click()
    
    
def get_text_by_selector(driver,selector,hidden=0):
    if hidden =='hidden' :
        data=driver.find_element_by_css_selector(selector)
        return (data.get_attribute("value"))
    data=driver.find_element_by_css_selector(selector)
    data=BeautifulSoup(data.text)
    return (data.text)

def Summary_tab_information(driver):

    data_tfoot=driver.find_element_by_xpath ( ("/html/body/div/div[2]/div/div[3]/div/div[2]/table/tfoot"))
    data_tfoot=data_tfoot.text.split('\n')
    
    
    COLUMN_NAMES_1=['total_product','total_shipping','total_price_without_tax','total_tax','total_price']
    Data_Summary_tab_part_1=pd.DataFrame(np.nan,index=[0],columns=COLUMN_NAMES_1)

    
    Data_Summary_tab_part_1['total_product']=float(data_tfoot[0].split('$')[1])
    Data_Summary_tab_part_1['total_shipping']=float(data_tfoot[1].split('$')[1])
    Data_Summary_tab_part_1['total_price_without_tax']=float(data_tfoot[2].split('$')[1])
    Data_Summary_tab_part_1['total_tax']=float(data_tfoot[3].split('$')[1])
    Data_Summary_tab_part_1['total_price']=float(data_tfoot[4].split('$')[1])
    
    
    data=driver.find_element_by_xpath ( ("/html/body/div/div[2]/div/div[3]/div/div[2]/table/tbody"))

    data=BeautifulSoup(data.get_attribute("outerHTML"))

    data = str(data.findAll("tr")).split('</tr>,')    
    
    COLUMN_NAMES_2=['Desc_title','Color','Size','SKU','cart_avail','cart_unit','cart_total','cart_quantity']

    Data_Summary_tab = pd.DataFrame(np.nan,index=range(len(data)),columns=COLUMN_NAMES_2)
    for i in range (0,len(data)):
        string='/html/body/div/div[2]/div/div[3]/div/div[2]/table/tbody/tr['+str(i+1)+']/td[5]/input[2]'
        qunty=driver.find_element_by_xpath ( (string)).get_attribute('value')
        working=BeautifulSoup(data[i])
        desc={}
        chunks=working.find('td',attrs={'class':'cart_description'}).text.strip()
        chunks=str(chunks).split('\n')
        remaning=chunks[1].split(',')[0].split(' ')
        Data_Summary_tab['Desc_title'].iloc[i]=chunks[0];Data_Summary_tab['SKU'].iloc[i]=remaning[2];
        Data_Summary_tab['Color'].iloc[i]=remaning[5];Data_Summary_tab['Size'].iloc[i]=chunks[1].split(',')[1].split(':')[1].strip()
        Data_Summary_tab['cart_avail'].iloc[i]=working.find('td',attrs={'class':'cart_avail'}).text.strip()
        Data_Summary_tab['cart_unit'].iloc[i]=working.find('td',attrs={'class':'cart_unit'}).text.strip()
        Data_Summary_tab['cart_total'].iloc[i]=working.find('td',attrs={'class':'cart_total'}).text.strip()
        Data_Summary_tab['cart_quantity'].iloc[i]=int(qunty)
        
    Data_Summary_tab['cart_unit'] = Data_Summary_tab['cart_unit'].str.replace('$', '')
    Data_Summary_tab['cart_unit']=  Data_Summary_tab['cart_unit'].astype(float)


    Data_Summary_tab['cart_total'] = Data_Summary_tab['cart_total'].str.replace('$', '')
    Data_Summary_tab['cart_total']=  Data_Summary_tab['cart_total'].astype(float) 
               
    return (Data_Summary_tab,Data_Summary_tab_part_1)

    

def String_comparsion(str_1,str_2):

    if str_1==str_2:
        return('Pass')
    else:
        return('Fail')
    

def Validation_order(Data_Summary_tab_part_1,Data_Summary_tab_part_2):
    
    COLUMN_NAMES=['Desc_title','Color','Size','SKU','cart_avail','cart_quantity','cart_total','total_product','total_price_without_tax','total_price']
    validation_Tap=pd.DataFrame(np.nan,index=[0],columns=COLUMN_NAMES)
    
    for i in range (0,len(Data_Summary_tab_part_2)):
        status=String_comparsion (Data_Summary_tab_part_2['Desc_title'].iloc[i].lower(),Data_Entry['order_wanted'].iloc[i].lower())
        if (status=='Pass'):        
            validation_Tap['Desc_title'].iloc[0]=status

        else:
            validation_Tap['Desc_title'].iloc[0]=status

        status=String_comparsion (Data_Summary_tab_part_2['Color'].iloc[i].lower(),Data_Entry['Color'].iloc[i].lower())

        if (status=='Pass'):
            validation_Tap['Color'].iloc[0]=status

        else:
            validation_Tap['Color'].iloc[0]=status    


        status=String_comparsion (Data_Summary_tab_part_2['Size'].iloc[i].lower(),Data_Entry['Size'].iloc[i].lower())

        if (status=='Pass'):
            validation_Tap['Size'].iloc[0]=status

        else:
            validation_Tap['Size'].iloc[0]=status


        status=String_comparsion (Data_Summary_tab_part_2['cart_avail'].iloc[i].lower(),Data_Entry['cart_avail'].iloc[i].lower())

        if (status=='Pass'):
            validation_Tap['cart_avail'].iloc[0]=status

        else:
            validation_Tap['cart_avail'].iloc[0]=status  


        if (Data_Summary_tab_part_2['cart_quantity'].iloc[i]==Data_Entry['cart_quantity'].iloc[i]):

            validation_Tap['cart_quantity'].iloc[0]=status

        else:
            validation_Tap['cart_quantity'].iloc[0]=status 

        cart_total=(Data_Summary_tab_part_2['cart_quantity'].iloc[i]*Data_Summary_tab_part_2['cart_unit'].iloc[i])
        if (Data_Summary_tab_part_2['cart_total'].iloc[i]==cart_total):

            validation_Tap['cart_total'].iloc[0]=status

        else:
            validation_Tap['cart_total'].iloc[0]=status


        if (Data_Summary_tab_part_1['total_product'].iloc[i]==Data_Summary_tab_part_2['cart_total'].iloc[i]):

            validation_Tap['total_product'].iloc[0]='Pass'

        else:
            validation_Tap['total_product'].iloc[0]='Fail'



        total_price_without_tax=(Data_Summary_tab_part_1['total_price_without_tax'].iloc[i]-Data_Summary_tab_part_1['total_tax'].iloc[i])
        if (Data_Summary_tab_part_1['total_price_without_tax'].iloc[i]==total_price_without_tax):

            validation_Tap['total_price_without_tax'].iloc[0]='Pass'

        else:
            validation_Tap['total_price_without_tax'].iloc[0]='Fail'


        Over_all_TOTAL=(Data_Summary_tab_part_1['total_price_without_tax'].iloc[i]+Data_Summary_tab_part_1['total_tax'].iloc[i])
        if (Data_Summary_tab_part_1['total_price_without_tax'].iloc[i]==Over_all_TOTAL):

            validation_Tap['total_price'].iloc[0]='Pass'

        else:
            validation_Tap['total_price'].iloc[0]='Fail'
            
        file_name='validation_Tap.csv'   
        validation_Tap.to_csv(file_name)
        
    return (validation_Tap,file_name)
    
    
def Errors_Validations(driver):
    Errors=get_text_by_selector(driver,'#center_column > div.alert.alert-danger').strip().replace('.','').split('\n')
    Errors_content=Errors[1:]
    Number_errors=Errors[0]
    return (Number_errors,Errors_content)  

def Error_Validations(driver,validation_case):
    Errors=get_text_by_selector(driver,'#center_column > div.alert.alert-danger').strip().replace('.','').split('\n')
    Errors_content=Errors[1:]
    Number_errors=Errors[0]
    Tab=validation_case
    for Errors_data in Errors_content:
        if validation_case=='Email_Required_validation':
                
                if 'email' and 'required' in Errors_data.lower():
                    Status='Pass'
                else :
                    Status='Fail'

        if validation_case=='Invalid_Email_validation':
                if 'email' and 'invalid' in Errors_data.lower():
                    Status='Pass'
                else :
                    Status='Fail'

        if validation_case=='Password_Required_validation':
                if 'password' and 'required' in Errors_data.lower():
                    Status='Pass'
                else :
                     Status='Fail'
                    
        if validation_case=='Authintication_Faild':
                if 'authentication' in Errors_data.lower():
                    Status='Pass'
                else :
                    Status='Fail'
    return (Number_errors,Errors_content,Status,Tab)

def Errors_test_cases(test_case):
    COLUMN_NAMES=['Email_Required_validation','Password_Required_validation','Invalid_Email_validation','Authintication_Faild']
    sign_in_Tap=pd.DataFrame(np.nan,index=[0],columns=COLUMN_NAMES)
    
    for i in range(0,len(test_case)):
        clear_entire_text(driver,'//*[@id="email"]')
        clear_entire_text(driver,'//*[@id="passwd"]')
        sleep(3)
        if test_case[i]=='Email_Required_validation':
            Clicking_on_element(driver,'//*[@id="SubmitLogin"]/span')
            Number_errors,Errors_content,Status,Tab=Error_Validations(driver,test_case[i])
            sign_in_Tap[test_case[i]]=Status
           
        elif test_case[i]=='Invalid_Email_validation':
            Inserting_Entry(driver,'//*[@id="email"]','sssssfefefem')
            Inserting_Entry(driver,'//*[@id="passwd"]','ssssss')
            Clicking_on_element(driver,'//*[@id="SubmitLogin"]/span')
            Number_errors,Errors_content,Status,Tab=Error_Validations(driver,test_case[i])
            sign_in_Tap[test_case[i]]=Status

        elif test_case[i]=='Password_Required_validation':
            
            Inserting_Entry(driver,'//*[@id="email"]','scscscs@fff.com')
            Clicking_on_element(driver,'//*[@id="SubmitLogin"]/span')
            Number_errors,Errors_content,Status,Tab=Error_Validations(driver,test_case[i])
            sign_in_Tap[test_case[i]]=Status

        elif test_case[i]=='Authintication_Faild':
            Inserting_Entry(driver,'//*[@id="email"]','sssss@gggg.com')
            Inserting_Entry(driver,'//*[@id="passwd"]','fefefefefe')
            Clicking_on_element(driver,'//*[@id="SubmitLogin"]/span')
            Number_errors,Errors_content,Status,Tab=Error_Validations(driver,test_case[i])
            sign_in_Tap[test_case[i]]=Status
        file_name='sign_in_Tap.csv'   
        sign_in_Tap.to_csv(file_name)
    return (sign_in_Tap,file_name)



def Create_email(driver,Personal_info,test_case_number):
    
    if Personal_info['gender'].iloc[test_case_number]=='male':
        Clicking_on_element(driver, '//*[@id="id_gender1"]')
    elif Personal_info['gender'].iloc[test_case_number]=='female' :
        Clicking_on_element(driver, '//*[@id="id_gender2"]')

    
    
    Select(driver.find_element_by_xpath('//*[@id="days"]')).select_by_value(str(int(Personal_info['day'].iloc[test_case_number])))
    Select(driver.find_element_by_xpath('//*[@id="months"]')).select_by_value(str(int(Personal_info['month'].iloc[test_case_number])))
    Select(driver.find_element_by_xpath('//*[@id="years"]')).select_by_value(str(int(Personal_info['year'].iloc[test_case_number])))

    Inserting_Entry(driver,'//*[@id="customer_firstname"]',Personal_info['customer_firstname'].iloc[test_case_number])
    Inserting_Entry(driver,'//*[@id="customer_lastname"]',Personal_info['customer_lastname'].iloc[test_case_number])
    Inserting_Entry(driver,'//*[@id="passwd"]',Personal_info['password'].iloc[test_case_number])
    
    Inserting_Entry(driver,'//*[@id="firstname"]',Personal_info['first_name'].iloc[test_case_number])
    Inserting_Entry(driver,'//*[@id="lastname"]',Personal_info['last_name'].iloc[test_case_number])

    
    Inserting_Entry(driver,'//*[@id="company"]',Personal_info['company'].iloc[test_case_number])

    Inserting_Entry(driver,'//*[@id="address1"]',Personal_info['address_1'].iloc[test_case_number])
#     Inserting_Entry(driver,'//*[@id="address2"]',Personal_info['address_2'].iloc[test_case_number])
    Inserting_Entry(driver,'//*[@id="city"]',Personal_info['city'].iloc[test_case_number])
    
    Select(driver.find_element_by_xpath('//*[@id="id_country"]')).select_by_visible_text(Personal_info['country'].iloc[test_case_number].title())
    
    Select(driver.find_element_by_xpath('//*[@id="id_state"]')).select_by_visible_text(Personal_info['state'].iloc[test_case_number].title())
    
    Inserting_Entry(driver,'//*[@id="postcode"]',int(Personal_info['postcode'].iloc[test_case_number])) 
    
    Inserting_Entry(driver,'//*[@id="other"]',Personal_info['Additional_information'].iloc[test_case_number])
    
    Inserting_Entry(driver,'//*[@id="phone"]',int(Personal_info['phone'].iloc[test_case_number]))
    
    Inserting_Entry(driver,'//*[@id="phone_mobile"]',int(Personal_info['phone_mobile'].iloc[test_case_number]))
    clear_entire_text(driver,'//*[@id="alias"]')
    Inserting_Entry(driver,'//*[@id="alias"]',Personal_info['reff'].iloc[test_case_number])
    
    
def proceed(driver):
    Click_on_element_by_displayed_text(driver,'Register')
    sleep(7)

    Clicking_on_element(driver,'//*[@id="center_column"]/form/p/button/span')
    sleep(7)

    Clicking_on_element(driver,'//*[@id="form"]/div/p[2]/a')

    sleep(7)
    Clicking_on_element(driver,'//*[@id="order"]/div[2]/div/div/a')
    sleep(5)
    Clicking_on_element(driver,'//*[@id="cgv"]')
    Clicking_on_element(driver,'//*[@id="form"]/p/button/span')


## Intialization

In [39]:
capabilities = DesiredCapabilities.CHROME
capabilities["goog:loggingPrefs"] = {"performance": "ALL"}  

driver = webdriver.Chrome(
    'C:/Users/C-LAB/Downloads/chromedriver_win32/chromedriver.exe',
    desired_capabilities=capabilities)

driver.set_window_size(1024, 768)
time=1


url='http://automationpractice.com/'

Data_Entry=pd.read_csv('Data_Entry.csv') 
Data_Entry.head()

,Search_words,Color,Size,SKU,cart_avail,order_wanted,cart_quantity
0,dress,blue,L,NaN,In stock,T-shirts > Faded Short Sleeve T-shirts,8


## Starting The Senario

In [40]:
getting_URL(driver,url)

Inserting_Entry(driver,'//*[@id="search_query_top"]',Data_Entry['Search_words'].iloc[0])

sleep(3)

Click_on_element_by_displayed_text(driver,Data_Entry['order_wanted'].iloc[0])
sleep(3)

Add_Quantity(Data_Entry['cart_quantity'].iloc[0],'Add')
sleep(3)
Choose_size(Data_Entry['Size'].iloc[0])
sleep(3)

Choose_Color(Data_Entry['Color'].iloc[0])
sleep(3)
Click_on_element_by_displayed_text(driver,'Add to cart')

sleep(10)

Clicking_on_element(driver,'//*[@id="layer_cart"]/div[1]/div[2]/div[4]/a/span')
 

Data_Summary_tab_part_2,Data_Summary_tab_part_1=Summary_tab_information(driver)
validation_Tap,file_name=Validation_order(Data_Summary_tab_part_1,Data_Summary_tab_part_2)
os.startfile(file_name)

file_name='Data_Summary_tab_part_1.csv'   
Data_Summary_tab_part_1.to_csv(file_name)
file_name='Data_Summary_tab_part_2.csv'   
Data_Summary_tab_part_2.to_csv(file_name)


sleep(2)
Clicking_on_element(driver,'//*[@id="center_column"]/p[2]/a[1]/span')
Test_cases=['Email_Required_validation','Password_Required_validation','Invalid_Email_validation','Authintication_Faild']
sleep(3)

sign_in_Tap,file_name=Errors_test_cases(Test_cases)
os.startfile(file_name)
sign_in_Tap.head()
sleep(3)
clear_entire_text(driver,'//*[@id="email_create"]')
sleep(3)

Inserting_Entry(driver,'//*[@id="email_create"]','Mohddam@gov.com')
sleep(1)
Clicking_on_element(driver,'//*[@id="SubmitCreate"]/span')
Personal_info=pd.read_csv('Personal_info.csv') 
Personal_info.head()
Create_email(driver,Personal_info,0)

proceed(driver)


In [41]:
Personal_info

,Unnamed: 0,gender,day,month,year,first_name,last_name,customer_firstname,customer_lastname,address_1,...,password,company,city,state,postcode,Additional_information,phone,phone_mobile,reff,country
0,0,male,12.0,5.0,1995.0,mohamed,elhajabdou,mohamed,elhajabdou,alex,...,mohamed@159,CRM,alex,florida,11114.0,No data,1.278770e+09,1.278770e+09,Mohamed@gov.com,united states
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
Data_Summary_tab_part_2

,Desc_title,Color,Size,SKU,cart_avail,cart_unit,cart_total,cart_quantity
0,Faded Short Sleeve T-shirts,Blue,L,demo_1,In stock,16.51,132.08,8.0


In [42]:
Data_Summary_tab_part_1

,total_product,total_shipping,total_price_without_tax,total_tax,total_price
0,132.08,2.0,134.08,0.0,134.08
